<a href="https://colab.research.google.com/github/4rt10m/scrapers/blob/main/TP_Scrape_Multi_site_Multi_term.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from time import sleep
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm


def soup2list(src, list_, attr=None):
    if attr:
        for val in src:
            list_.append(val[attr])
    else:
        for val in src:
            list_.append(val.get_text())

# Define the URLs and search terms
urls = ['www.worldremit.com']  # Add more URLs as needed
search_terms = ['international', 'fees,cost,fee,costs', 'exchange rate,exchange rates','speed','limit','feature,features','safe,security,safety,secure','ease,easy','support,customer support']  # Add more search terms as needed

from_page = 2
to_page = 6

# Iterate over each URL
for company in tqdm(urls, desc="Processing URLs"):
    # Iterate over each search term for the current URL
    for search in tqdm(search_terms, desc=f"Processing search terms for {company}", leave=False):
        users = []
        ratings = []
        locations = []
        dates = []
        reviews = []

        # Scrape the specified range of pages
        for i in tqdm(range(from_page, to_page + 1), desc="Processing pages", leave=False):
            result = requests.get(fr"https://www.trustpilot.com/review/{company}?page={i}&search={search}")
            soup = BeautifulSoup(result.content, features="lxml")

            # Temporary lists to hold the scraped data for the current page
            temp_users = []
            temp_locations = []
            temp_dates = []
            temp_ratings = []
            temp_reviews = []

            soup2list(soup.find_all('span', {'class': 'typography_heading-xxs__QKBS8 typography_appearance-default__AAY17'}), temp_users)
            soup2list(soup.find_all('div', {'class': 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__Fo_ua'}), temp_locations)
            soup2list(soup.find_all('div', {'class': 'styles_reviewHeader__iU9Px'}), temp_dates)
            soup2list(soup.find_all('div', {'class': 'styles_reviewHeader__iU9Px'}), temp_ratings, attr='data-service-review-rating')
            soup2list(soup.find_all('div', {'class': 'styles_reviewContent__0Q2Tg'}), temp_reviews)

            # Ensure all temporary lists are of the same length
            max_length = max(len(temp_users), len(temp_locations), len(temp_dates), len(temp_ratings), len(temp_reviews))

            temp_users.extend(["BLANK"] * (max_length - len(temp_users)))
            temp_locations.extend(["BLANK"] * (max_length - len(temp_locations)))
            temp_dates.extend(["BLANK"] * (max_length - len(temp_dates)))
            temp_ratings.extend(["BLANK"] * (max_length - len(temp_ratings)))
            temp_reviews.extend(["BLANK"] * (max_length - len(temp_reviews)))

            users.extend(temp_users)
            locations.extend(temp_locations)
            dates.extend(temp_dates)
            ratings.extend(temp_ratings)
            reviews.extend(temp_reviews)

            # To avoid throttling
            sleep(1)

        # Add columns for the URL and search term
        urls_column = [company] * len(users)
        search_terms_column = [search] * len(users)

        # Convert the data to a DataFrame
        review_data = pd.DataFrame(
            {
                'URL': urls_column,
                'Search Term': search_terms_column,
                'Username': users,
                'Location': locations,
                'Date': dates,
                'Content': reviews,
                'Rating': ratings
            }
        )

        # Create a filename for the current URL and search term
        filename = f"REVIEW - {company.replace('www.', '').replace('.com', '')} - {search}.csv"

        # Save the DataFrame to a CSV file
        review_data.to_csv(filename, index=False)

Processing search terms for www.worldremit.com:   0%|          | 0/9 [00:00<?, ?it/s]

Processing pages:   0%|          | 0/5 [00:00<?, ?it/s]

Processing pages:  20%|██        | 1/5 [00:03<00:12,  3.13s/it]

Processing pages:  40%|████      | 2/5 [00:06<00:09,  3.12s/it]

Processing pages:  60%|██████    | 3/5 [00:08<00:05,  2.94s/it]

Processing pages:  80%|████████  | 4/5 [00:11<00:02,  2.63s/it]

Processing pages: 100%|██████████| 5/5 [00:14<00:00,  2.77s/it]

                                                               
Processing search terms for www.worldremit.com:  11%|█         | 1/9 [00:14<01:53, 14.20s/it]

Processing pages:   0%|          | 0/5 [00:00<?, ?it/s]

Processing pages:  20%|██        | 1/5 [00:02<00:11,  2.93s/it]

Processing pages:  40%|████      | 2/5 [00:05<00:08,  2.77s/it]

Processing pages:  60%|██████    | 3/5 [00:08<00:05,  2.75s/it]

Processing pages:  80%|████████  | 4/5 [00:11<00:02,  2.74s/it]

Processing pages: 100%|██████████| 5/5 [00:14<00:00,  2